In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


# **Importing the tools**

In [3]:
import re
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformer import Transformer

## Importing the **Data**

In [4]:
with open('/content/drive/MyDrive/dataset_train.txt','r') as f:
    data = f.read()
print(data[:100])


Question: how do i stop smoking now
Answer: stopping smoking is about will power and being steadfas


# **Preprocess**

In [5]:
input,output = [],[]
qa_pairs = re.split(r"Question: |Answer: ", data)[1:]

# Print the separated question-answer pairs
for i in range(0, len(qa_pairs), 2):
    question = qa_pairs[i].strip()
    answer = qa_pairs[i + 1].strip()
    if question in input:
        continue
    else:
        input.append(''.join(question.split('.')[:1]))
        output.append(''.join(answer.split('.')[:3]))

print(input[:100])
print(output[:100])

['how do i stop smoking now', 'i had a tubaligation 4 years ago and also have a minor case of endometriosis i am only 27 and my period stopped completely 3 months ago no more spotting in between nothing i am not stressed no change in daily habits diet weight nothing all urine pregnancy tests have been negative what could be wrong with me?', 'could extra caffeine consumption be a cause of mild unilateral breast tenderness?', 'hello- i am a 24 year old female 5"4 & 115 lb & was diagnosed with pcos at 19', 'i was wanting to know if you could tell me if its possible for a woman to go through all their pregnancy without knowing they are pregnant and have periods and ngative pregnancy tests? iwas watching the show called "i didnt know i was pregnant" and wanted to know how real and accurate this show is and how somthing like that could happen?', 'i have been taking methadose and oxycotin illeagely for over a year now', 'i need help i drink every night and think that it has something to do wi

In [6]:
input = input[:2000]
output = output[:2000]

In [7]:
input[1999],output[1999]

('i’ve neck muscles strain that caused a temporomandibular joint disorders (tmj tmd)?',
 'this sounds like trigeminal neuralgia symptoms do you often chew gums? your headache may be related to neck muscle spasm yes…but it is not all caused by neck strain <|endoftext|>')

In [8]:
output = [i.replace('<|endoftext|>','') for i in output]
tokens = ""

In [9]:
for i in input:
    for j in i:
        if j in tokens:
            continue
        else:
            tokens += j

for i in output:
    for j in i:
        if j in tokens:
            continue
        else:
            tokens += j

print(tokens)

how distpmkngaubl4yervcf273?x-5"&19()'0!j\:6<_>qz8/;%’–…“”‘+éâ¦î±​̇🙂،=—℅


In [10]:
START_TOKEN = '<START>'
END_TOKEN = '<END>'
PADDING_TOKEN = '<PAD>'
tokens = list(tokens)
tokens.append(START_TOKEN)
tokens.append(END_TOKEN)
tokens.append(PADDING_TOKEN)
print(tokens)

['h', 'o', 'w', ' ', 'd', 'i', 's', 't', 'p', 'm', 'k', 'n', 'g', 'a', 'u', 'b', 'l', '4', 'y', 'e', 'r', 'v', 'c', 'f', '2', '7', '3', '?', 'x', '-', '5', '"', '&', '1', '9', '(', ')', "'", '0', '!', 'j', '\\', ':', '6', '<', '_', '>', 'q', 'z', '8', '/', ';', '%', '’', '–', '…', '“', '”', '‘', '+', 'é', 'â', '\x80', '\x99', '\x98', '\x94', '\x93', '\x9c', '\x9d', '¦', 'î', '±', '\u200b', '̇', '🙂', '،', '=', '—', '℅', '<START>', '<END>', '<PAD>']


In [11]:
lang_to_index = {v:k for k,v in enumerate(tokens)}
index_to_lang = {k:v for k,v in enumerate(tokens)}
print(lang_to_index)
print(index_to_lang)

{'h': 0, 'o': 1, 'w': 2, ' ': 3, 'd': 4, 'i': 5, 's': 6, 't': 7, 'p': 8, 'm': 9, 'k': 10, 'n': 11, 'g': 12, 'a': 13, 'u': 14, 'b': 15, 'l': 16, '4': 17, 'y': 18, 'e': 19, 'r': 20, 'v': 21, 'c': 22, 'f': 23, '2': 24, '7': 25, '3': 26, '?': 27, 'x': 28, '-': 29, '5': 30, '"': 31, '&': 32, '1': 33, '9': 34, '(': 35, ')': 36, "'": 37, '0': 38, '!': 39, 'j': 40, '\\': 41, ':': 42, '6': 43, '<': 44, '_': 45, '>': 46, 'q': 47, 'z': 48, '8': 49, '/': 50, ';': 51, '%': 52, '’': 53, '–': 54, '…': 55, '“': 56, '”': 57, '‘': 58, '+': 59, 'é': 60, 'â': 61, '\x80': 62, '\x99': 63, '\x98': 64, '\x94': 65, '\x93': 66, '\x9c': 67, '\x9d': 68, '¦': 69, 'î': 70, '±': 71, '\u200b': 72, '̇': 73, '🙂': 74, '،': 75, '=': 76, '—': 77, '℅': 78, '<START>': 79, '<END>': 80, '<PAD>': 81}
{0: 'h', 1: 'o', 2: 'w', 3: ' ', 4: 'd', 5: 'i', 6: 's', 7: 't', 8: 'p', 9: 'm', 10: 'k', 11: 'n', 12: 'g', 13: 'a', 14: 'u', 15: 'b', 16: 'l', 17: '4', 18: 'y', 19: 'e', 20: 'r', 21: 'v', 22: 'c', 23: 'f', 24: '2', 25: '7', 26: '

In [12]:
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length Kannada: {np.percentile([len(x) for x in output], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in input], PERCENTILE)}" )

97th percentile length Kannada: 456.03
97th percentile length English: 132.0


In [13]:
max_sequence_length = 460

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(output)):
    a, q = output[index], input[index]
    if is_valid_length(a, max_sequence_length) \
      and is_valid_length(q, max_sequence_length) \
      and is_valid_tokens(a, tokens):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(output)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 2000
Number of valid sentences: 1940


In [14]:
input = [input[i] for i in valid_sentence_indicies]
output = [output[i] for i in valid_sentence_indicies]

In [15]:
len(input)
print(input[0])
print(output[0])

how do i stop smoking now
stopping smoking is about will power and being steadfast you can stop safely by having bupropion or nicotine patch cover initially in consult with a doctor contact an addiction clinic near you


# **Lets Create a Dataset**

In [16]:
class FitFuelDataset(Dataset):
    def __init__(self,input=input,output=output):
        self.input = input
        self.output = output
    def __len__(self):
        return len(self.input)
    def __getitem__(self,i):
        return self.input[i],self.output[i]

In [17]:
Train_data = FitFuelDataset()
len(Train_data)

1940

# **Lets make DataLoader**

In [18]:
bs = 4
Train_loader = DataLoader(Train_data,batch_size=bs,shuffle=True)
len(Train_loader)

485

In [23]:
d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 460
kn_vocab_size = len(tokens)

transformer = Transformer(d_model,
                          ffn_hidden,
                          num_heads,
                          drop_prob,
                          num_layers,
                          max_sequence_length,
                          kn_vocab_size,
                          lang_to_index,
                          lang_to_index,
                          START_TOKEN,
                          END_TOKEN,
                          PADDING_TOKEN)

In [24]:
criterian = nn.CrossEntropyLoss(ignore_index=lang_to_index[PADDING_TOKEN],
                                reduction='none')

for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [25]:
NEG_INFTY = -1e9

def create_masks(eng_batch, kn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, kn_sentence_length = len(eng_batch[idx]), len(kn_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      kn_chars_to_padding_mask = np.arange(kn_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, kn_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, kn_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, kn_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [ ]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 200

for epoch in range(num_epochs):
    total_loss = 0
    print(f"Epoch {epoch}")
    iterator = iter(Train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        q, a = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(q, a)
        optim.zero_grad()
        a_predictions = transformer(q,
                                     a,
                                     encoder_self_attention_mask.to(device),
                                     decoder_self_attention_mask.to(device),
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(a, start_token=False, end_token=True)
        loss = criterian(
            a_predictions.view(-1, kn_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == lang_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        total_loss += loss.item()
        loss.backward()
        optim.step()
    total_loss /= len(iterator)
        #train_losses.append(loss.item())
    if batch_num % 1 == 0:
            print(f"Iteration {batch_num} : {total_loss}")
            print(f"Q: {q[0]}")
            print(f"ans: {a[0]}")
            kn_sentence_predicted = torch.argmax(a_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in kn_sentence_predicted:
              if idx == lang_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_lang[idx.item()]
            print(f"Prediction: {predicted_sentence}")


    transformer.eval()
    a = ("",)
    q = ("how do i stop smoking now",)
    for word_counter in range(max_sequence_length):
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(q, a)
        predictions = transformer(q,
                                          a,
                                          encoder_self_attention_mask.to(device),
                                          decoder_self_attention_mask.to(device),
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
        next_token_prob_distribution = predictions[0][word_counter] # not actual probs
        next_token_index = torch.argmax(next_token_prob_distribution).item()
        next_token = index_to_lang[next_token_index]
        a = (a[0] + next_token, )
        if next_token == END_TOKEN:
            break

    print(f"Evaluation translation (how do i stop smoking now) : {a}")
    print("-------------------------------------------")

Epoch 0
Iteration 484 : 0.4321472952046345
Q: i have migraine headaches blurred vision disoriented slurred speech fever weakness nothing is helping me?
ans: cryselle long term use can cause your migraines also long term use of pain killers can cause morning headaches i recommend asking your gynaecologist to switch cryselle good luck 
Prediction: hrysalle mong torm usu man causeswour megraines aroo cong term us agf pain kille s can cause morning keadache  i recommend asking aour gynaecolorist to switch crasille good luck 
Evaluation translation (how do i stop smoking now) : ('stopping smoking is about will power and being steadfast you can stop safely by having bupropion or nicotine patch cover initially the deally be done contact and be done pathologist conths contact and blood with contact with consult about is (a doctor consult and keep then becover a week a making you<END>',)
-------------------------------------------
Epoch 1
Iteration 484 : 0.4296435306981667
Q: i have heard that 

In [27]:
torch.save(transformer.state_dict(),'prototype_400.pth')

In [ ]:
transformer.eval()
def translate(text):
  q = (text,)
  a = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(q, a)
    predictions = transformer(q,
                              a,
                              encoder_self_attention_mask.to(device),
                              decoder_self_attention_mask.to(device),
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_lang[next_token_index]
    a = (a[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return a[0]